In [ ]:
import json

from pyproj import CRS, Transformer
from specklepy.api.client import SpeckleClient
from specklepy.core.api import operations
from specklepy.transports.server import ServerTransport

# Leapfrog Weka Hills Demo Model from Speckle to IFC 5

In [ ]:
weka_hills_speckle_project_id = "7a489ac0d4"
ifc5_geo_dict = {"name": "Alluvium", "ifc5_ref": "Ne4ef1ea1101a484183bb8a716924702e"}
# ifc5_geo_dict = {"name": "Siltstone", "ifc5_ref": "N4586ef982fd34faab3de532a7a43544d"}
# ifc5_geo_dict = {"name": "Basement", "ifc5_ref": "N450a532f5e414d609856a8a50b6a94ad"}

In [ ]:
speckle_client = SpeckleClient(host="app.speckle.systems")
transport = ServerTransport(
    stream_id=weka_hills_speckle_project_id, client=speckle_client
)

In [ ]:
weka_hills_speckle_models = speckle_client.model.get_models(
    project_id=weka_hills_speckle_project_id
)

geology_models_w_versions = {}
for model in weka_hills_speckle_models.items:
    if "geo/geology-model" in model.name:
        geological_unit = str(model.name).replace("geo/geology-model/", "")
        speckle_model = speckle_client.model.get_with_versions(
            model_id=model.id,
            project_id=weka_hills_speckle_project_id,
        )
        geology_models_w_versions[geological_unit] = speckle_model
        print(f"{geological_unit}: {model.id}")

In [ ]:
model_data = operations.receive(
    geology_models_w_versions[ifc5_geo_dict["name"].lower()]
    .versions.items[0]
    .referencedObject,
    transport,
)

In [ ]:
speckle_faces = model_data.elements[0].elements[0].elements[0].elements[0].faces
face_vertex_counts = []
face_vertex_indices = []
i = 0
while i < len(speckle_faces):
    vertex_count = speckle_faces[i]
    face_vertex_counts.append(vertex_count)
    face_vertex_indices.extend(speckle_faces[i + 1 : i + 1 + vertex_count])
    i += 1 + vertex_count

speckle_vertices = model_data.elements[0].elements[0].elements[0].elements[0].vertices
points = [speckle_vertices[i : i + 3] for i in range(0, len(speckle_vertices), 3)]

In [ ]:
ifc5_usdgeom = [
    {
        "def": "over",
        "name": f"{ifc5_geo_dict['ref']}_Body",
        "attributes": {
            "UsdGeom:Mesh": {
                # "faceVertexCounts": face_vertex_counts,
                "faceVertexIndices": face_vertex_indices,
                "points": points,
            }
        },
    }
]

with open(
    f"WekaHills_GeologyModel_{ifc5_geo_dict['name']}_UsdGeom.ifcx", "w"
) as json_file:
    json.dump(ifc5_usdgeom, json_file)

For testing purposes, one can create an `.obj` file from the Speckle geometry as follows:

In [ ]:
with open(f"{geological_unit}Mesh.obj", "w") as obj_file:
    for point in points:
        obj_file.write(f"v {point[0]} {point[1]} {point[2]}\n")
    for i in range(0, len(face_vertex_indices), 3):
        # OBJ uses 1-based indexing, so add 1 to the indices
        obj_file.write(
            f"f {face_vertex_indices[i] + 1} {face_vertex_indices[i + 1] + 1} {face_vertex_indices[i + 2] + 1}\n"
        )

# CRS Objects in IFC 5

Coordinate Reference Systems (CRS') can be defined using the Well-Known Text representation of Coordinate Reference Systems (WKT-CRS | [Wikipedia](http://en.wikipedia.org/wiki/Well-known_text_representation_of_coordinate_reference_systems) | [OGC Specification](https://docs.ogc.org/is/18-010r7/18-010r7.html)). This representation of CRS' is also an ISO standard: ISO 19162:2019.

Notes from the Wikipedia WKT-CRS "Version history" section:

>  Confusingly, the original 2015 "WKT 2" standard has a version number 1 for the new, stricter WKT-CRS specification. A newer revision called WKT-CRS 2 was published in 2018, with the ISO version being ISO 19162:2019.

> ESRI vs OGC  
> ... some databases differentiate between "OGC WKT" and "ESRI WKT" representations. The problem is largely resolved in WKT[-CRS] 2, as it is better-defined.

These differences in how WKT strings are also clear from the export section when looking up a CRS on [epsg.io](https://epsg.io). For example, have a look at the differences between the "OGC WKT", "OGC WKT 2" and "ESRI WKT" results in the export section at the bottom of the [epsg.io/2193](https://epsg.io/2193) page.

In [ ]:
weka_hills_crs = CRS(2193)

In [ ]:
ifc5_wkt_crs = [
    {
        "def": "def",
        "name": "CRS_EPSG:2193",
        "type": "OGC:CRS",
        "attributes": {
            "WKT-CRS": {
                "ISO": "ISO 19162:2019",
                "wkt": weka_hills_crs.to_wkt(),
                "name": weka_hills_crs.name,
                "authority_id": f"EPSG:{weka_hills_crs.to_authority()[1]}",
                "units": "m",
                "offset_x": 0,
                "offset_y": 0,
                "offset_z": 0,
                "rotation": 0,
            }
        },
    },
    {
        "def": "over",
        "name": "Nad68cd145a6b43fdb0232e0b11a6c7bc",
        "comment": "Assigns CRS_EPSG:2193 to the instance of borehole WH_009",
        "attributes": {"crs": {"ref": "</CRS_EPSG:2193>"}},
    },
    {
        "def": "over",
        "name": "N996ff9e7b86240f898d4d038ab311d1b",
        "comment": "Assigns CRS_EPSG:2193 to the instance of borehole WH_012",
        "attributes": {"crs": {"ref": "</CRS_EPSG:2193>"}},
    },
    {
        "def": "over",
        "name": "N25dbbae886194be6b469e4f17f0b605c",
        "comment": "Assigns CRS_EPSG:2193 to the instance of the GeologyModel",
        "attributes": {"crs": {"ref": "</CRS_EPSG:2193>"}},
    },
]

with open("WekaHills_CRS_EPSG2193.ifcx", "w") as json_file:
    json.dump(ifc5_wkt_crs, json_file, indent=2)

Longitude is the x-coordinate or east-west coordinate on earth expressed in 0 to 180 degrees east or west of the Greenwich prime meridian or in -180 to 180 degrees relative to the Greenwich prime meridian.

Latitude is the y-coordinate or north-south coordinate on earth expressed in 0 to 90 north or south of the equator or in -90 to 90 degrees relative to the equator.

Historically, coordinates on earth are denoted (Latitude, Longitude), because it was easier to determine the north-south position than the east-west position. This because the north-south position can be determined from the sun, stars and other heavenly bodies, whereas one need to accurately know the time in order to determine the east-west position.

Nowadays, especially on the web, it's common to write coordinates on earth in [Longitude, Latitude], because we commonly write down coordinates [x, y].

In [ ]:
transformer = Transformer.from_crs(weka_hills_crs, 4326, always_xy=True)
custom_lon, custom_lat = transformer.transform(1166250.104, 4888836.859)
custom_crs_wkt = f'PROJCS["SpeckleCRS_latlon_{custom_lat}_{custom_lon}", GEOGCS["GCS_WGS_1984", DATUM["D_WGS_1984", SPHEROID["WGS_1984", 6378137.0, 298.257223563]], PRIMEM["Greenwich", 0.0], UNIT["Degree", 0.0174532925199433]], PROJECTION["Transverse_Mercator"], PARAMETER["False_Easting", 0.0], PARAMETER["False_Northing", 0.0], PARAMETER["Central_Meridian", {custom_lon}], PARAMETER["Scale_Factor", 1.0], PARAMETER["Latitude_Of_Origin", {custom_lat}], UNIT["Meter", 1.0]]'